In [1]:
from keras.datasets import cifar10
from keras.layers import Input, Dense,Conv2D,MaxPooling2D,UpSampling2D,BatchNormalization
from keras.models import Model,Sequential
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train.shape

(50000, 32, 32, 3)

In [4]:
x_train_flat = x_train.reshape(-1,3072)

In [5]:
x_train_flat.shape

(50000, 3072)

In [6]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [7]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(x_train_flat)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [8]:
x_test_flat = x_test.reshape(-1,3072)

In [9]:
train_img_pca = pca.transform(x_train_flat)
test_img_pca = pca.transform(x_test_flat)

In [10]:
train_img_pca = train_img_pca.reshape(-1, 32,32,3)
test_img_pca = test_img_pca.reshape(-1, 32,32,3)

In [11]:
from keras.layers import Conv1D,Embedding,Dropout,Flatten
from tensorflow.keras.constraints import max_norm

model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32,32,3)))
model1.add(Dropout(0.2))
model1.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(512, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(y_test.shape[1], activation='softmax'))
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

In [14]:
from keras.optimizers import RMSprop
model1.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model1.fit(train_img_pca, y_train,batch_size=128,epochs=5,verbose=1,
                    validation_data=(test_img_pca, y_test))

Epoch 1/5
391/391 [==============================] - 178s 455ms/step - loss: 5.3482 - accuracy: 0.2129 - val_loss: 1.8966 - val_accuracy: 0.3336
Epoch 2/5
391/391 [==============================] - 205s 525ms/step - loss: 1.9369 - accuracy: 0.3011 - val_loss: 1.7719 - val_accuracy: 0.3746
Epoch 3/5
391/391 [==============================] - 155s 396ms/step - loss: 1.8387 - accuracy: 0.3374 - val_loss: 1.7181 - val_accuracy: 0.3934
Epoch 4/5
391/391 [==============================] - 158s 403ms/step - loss: 1.7830 - accuracy: 0.3635 - val_loss: 1.6578 - val_accuracy: 0.4133
Epoch 5/5
391/391 [==============================] - 159s 407ms/step - loss: 1.7591 - accuracy: 0.3707 - val_loss: 1.6512 - val_accuracy: 0.4194


In [ ]:
train_img_pca.shape,y_train.shape,test_img_pca.shape,y_test.shape